In [1]:
import time
from qumulo.rest_client import RestClient

In [2]:
def get_throughput_data(rc):
    results = {}
    for node in rc.cluster.list_nodes():
        node_id = node['id']
        ns = rc.network.get_network_status_v2(1,node_id)
        results[node_id] = ns['interface_details']
    return results


def sizeof_fmt(num, suffix='B'):
    for unit in ['','K','M','G','T','P','E','Z']:
        if abs(num) < 1000.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1000.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

# Throughput by node, over time

In [4]:
rc = RestClient("product", 8000)
rc.login("admin", "admin")

interval_count = 5
interval_time_seconds = 5
previous_result = None

for i in range(0, interval_count+1):
    current_result = get_throughput_data(rc)
    if previous_result is not None:
        for node_id, current_data in current_result.iteritems():
            time_diff = (int(current_data['timestamp']) - int(previous_result[node_id]['timestamp'])) / 1000
            sent_diff = int(current_data['bytes_sent']) - int(previous_result[node_id]['bytes_sent'])
            recv_diff = int(current_data['bytes_received']) - int(previous_result[node_id]['bytes_received'])
            print("Node:%3s  --  sent: %s/s  --  recv: %s/s" % ( node_id,
                                                            sizeof_fmt(sent_diff / time_diff),
                                                            sizeof_fmt(recv_diff / time_diff),
                                    ))
        print("") # line break
    previous_result = current_result
    time.sleep(interval_time_seconds)


Node:  1  --  sent: 19.5KB/s  --  recv: 40.5KB/s
Node:  2  --  sent: 6.1KB/s  --  recv: 38.1KB/s
Node:  3  --  sent: 11.3KB/s  --  recv: 18.9KB/s
Node:  4  --  sent: 8.6KB/s  --  recv: 20.6KB/s

Node:  1  --  sent: 38.6KB/s  --  recv: 103.7KB/s
Node:  2  --  sent: 64.9KB/s  --  recv: 77.2KB/s
Node:  3  --  sent: 238.0KB/s  --  recv: 114.7KB/s
Node:  4  --  sent: 17.0KB/s  --  recv: 106.4KB/s

Node:  1  --  sent: 9.4KB/s  --  recv: 30.8KB/s
Node:  2  --  sent: 3.7KB/s  --  recv: 16.8KB/s
Node:  3  --  sent: 6.3KB/s  --  recv: 21.5KB/s
Node:  4  --  sent: 5.9KB/s  --  recv: 17.3KB/s

Node:  1  --  sent: 16.7KB/s  --  recv: 29.7KB/s
Node:  2  --  sent: 6.8KB/s  --  recv: 24.0KB/s
Node:  3  --  sent: 10.7KB/s  --  recv: 25.3KB/s
Node:  4  --  sent: 8.2KB/s  --  recv: 62.8KB/s

Node:  1  --  sent: 3.2MB/s  --  recv: 3.2MB/s
Node:  2  --  sent: 14.8KB/s  --  recv: 441.4KB/s
Node:  3  --  sent: 244.4KB/s  --  recv: 1.7MB/s
Node:  4  --  sent: 16.3KB/s  --  recv: 1.3MB/s

